In [1]:
from pyspark.sql import HiveContext
from pyspark import SparkContext
from pyspark.sql.functions import *   #importar todas as funções do pyspark sql
from pyspark.sql.types import *       #importar as conversoes
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [2]:
sqlContext = HiveContext(sc)

In [3]:
sqlContext.sql("show databases").show()

+------------+
|databaseName|
+------------+
|       covid|
|     default|
+------------+



In [4]:
sqlContext.sql("use covid")

DataFrame[]

In [5]:
sqlContext.sql("show tables").show()

+--------+-----------------+-----------+
|database|        tableName|isTemporary|
+--------+-----------------+-----------+
|   covid|            covid|      false|
|   covid|   covidfirstview|      false|
|   covid|covid_partitioned|      false|
+--------+-----------------+-----------+



In [6]:
covid = sqlContext.sql('select * from covid_partitioned')

In [7]:
covidDF = spark.read.table('covid_partitioned')
covidDF.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- coduf: integer (nullable = true)
 |-- codmun: integer (nullable = true)
 |-- codregiaosaude: integer (nullable = true)
 |-- nomeregiaosaude: string (nullable = true)
 |-- data: string (nullable = true)
 |-- semanaepi: integer (nullable = true)
 |-- populacaotcu219: integer (nullable = true)
 |-- casosacumulados: integer (nullable = true)
 |-- casosnovos: integer (nullable = true)
 |-- obitosacumulados: integer (nullable = true)
 |-- obitosnovos: integer (nullable = true)
 |-- recuperadosnovos: integer (nullable = true)
 |-- emacompanhamentonovos: integer (nullable = true)
 |-- interiormetropolitana: integer (nullable = true)
 |-- municipio: string (nullable = true)



In [8]:
covidFirstDF = covidDF.agg((max("casosnovos")-max("emacompanhamentonovos")).alias("Emacompanhamento"), max("emacompanhamentonovos").alias("CasosRecuperados"))
covidFirstDF.show()

+----------------+----------------+
|Emacompanhamento|CasosRecuperados|
+----------------+----------------+
|         1592369|        17262646|
+----------------+----------------+



In [9]:
covidFirstDF.write.saveAsTable("covidFirstView")

AnalysisException: 'Table `covidFirstView` already exists.;'

In [10]:
covidSecDF = covidDF.agg(max("casosnovos").alias("Acumulado"), (max("obitosnovos")-last("casosnovos")).alias("CasosNovos"), format_number((max("casosnovos")*max("populacaotcu219")/100000),2).alias("Incidencia"))
covidSecDF.show()

+---------+----------+----------+
|Acumulado|CasosNovos|Incidencia|
+---------+----------+----------+
| 18855015|     92184|  9,993.16|
+---------+----------+----------+



In [ ]:
covidSecDF.write.parquet('/user/marcel/data/covid/covidSecView_teste', compression="snappy")

In [11]:
covidThDF= covidDF.agg(max("obitosnovos").alias("ObitosAcumulados"), (last("obitosacumulados")).alias("CasosNovos"), format_number((max("obitosnovos")/max("casosnovos"))*100,2).alias("Letalidade") ,format_number((max("obitosnovos")*max("populacaotcu219")/100000),2).alias("Mortalidade"))
covidThDF.show()

+----------------+----------+----------+-----------+
|ObitosAcumulados|CasosNovos|Letalidade|Mortalidade|
+----------------+----------+----------+-----------+
|          526892|       594|      2.79|     279.25|
+----------------+----------+----------+-----------+



In [ ]:
#covidThDF.write.json('/user/marcel/data/covid/covidThView1')
covidThDF.select(from_json("covidThView1").writeStream)

In [17]:
covidSparkDF = covidDF.groupBy("regiao").agg(max("casosnovos").alias("Casos"), max("obitosnovos").alias("Obitos"), format_number((max("casosnovos")*max("populacaotcu219")/100000),2).alias("Incidencia"), format_number((max("obitosnovos")*max("populacaotcu219")/100000),2).alias("Mortalidade"), last("nomeregiaosaude").alias("Data")).show()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: functions does not exist in the JVM